# 策略評估(Policy evaluation)

In [1]:
from IPython.core.debugger import set_trace
import numpy as np
import pprint
import sys
from lib.envs.gridworld import GridworldEnv

In [2]:
# 載入遊戲
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
# 策略評估函數
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    policy: [S, A] shaped matrix representing the policy.
    env.P：狀態轉移機率矩陣
    env.P[s][a]：(prob, next_state, reward, done) 陣列
    env.nS：狀態總數
    env.nA：行動總數
    theta: 狀態值函數變動 <= theta，即停止進行。
    discount_factor: 獎勵折扣率(Gamma)
    
    傳回【狀態值函數】
    """
    # 一開始【狀態值函數】均為 0
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # 執行每一種可能的行動
            for a, action_prob in enumerate(policy[s]):
                # 行動可能到達的每一種狀態
                for  prob, next_state, reward, done in env.P[s][a]:
                    # 計算【狀態值函數】期望值，參考圖三. 狀態值函數公式
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # 找出每一個位置的這一輪與上一輪狀態值函數差異最大者
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # 最大差異 < 事先設定值 theta，則停止評估
        if delta < theta:
            break
    return np.array(V)

## 測試

In [4]:
# 隨機策略，每一個位置往上、下、左、右走的機率均為 0.25。
random_policy = np.ones([env.nS, env.nA]) / env.nA
# 策略評估
v = policy_eval(random_policy, env)

## 顯示狀態值函數

In [5]:
print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Value Function:
[  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569
   0.        ]

Reshaped Grid Value Function:
[[  0.         -13.99993529 -19.99990698 -21.99989761]
 [-13.99993529 -17.9999206  -19.99991379 -19.99991477]
 [-19.99990698 -19.99991379 -17.99992725 -13.99994569]
 [-21.99989761 -19.99991477 -13.99994569   0.        ]]



## 顯示最後結果

In [6]:
# 最後結果
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)